<span>
<img src="http://ash.readthedocs.io/en/latest/_static/ash.png" width="260px" align="right"/>
</span>
<span>
<b>Author:</b> <a href="https://andreafailla.github.io">Andrea Failla</a><br/>
<b>Python version:</b>  3.9<br/>
<b>ASH version:</b>  1.0.0<br/>
<b>Last update:</b> November 2025
</span>

<a id="hypergraph-segregation-analysis"></a>
# Hypergraph Segregation Analysis

In [3]:
import sys
sys.path.insert(0, '../')

<a id="table-of-contents"></a>
# Table of Contents

  - [Introduction](#introduction)
  - [What is Segregation in Hypergraphs?](#what-is-segregation-in-hypergraphs)
  - [Random Walk Hypergraph Segregation (RWHS)](#random-walk-hypergraph-segregation-rwhs)
    - [Meet-wise RWHS](#meet-wise-rwhs)
    - [Jump-wise RWHS](#jump-wise-rwhs)
  - [Static RWHS](#static-rwhs)
    - [Basic Example](#basic-example)
    - [Comparing Methods](#comparing-methods)
    - [Effect of Random Walk Parameters](#effect-of-random-walk-parameters)
  - [Temporal RWHS](#temporal-rwhs)
  - [Practical Examples](#practical-examples)
    - [Measuring Attribute Mixing](#measuring-attribute-mixing)
    - [Comparing Segregation Across Time](#comparing-segregation-across-time)
    - [Parameter Tuning](#parameter-tuning)
  - [References](#references)

<a id="introduction"></a>
## Introduction

**Segregation** is a fundamental concept in social network analysis that measures how individuals with similar attributes tend to group together. While traditional segregation measures focus on pairwise relationships (edges), hypergraphs allow us to study segregation in **higher-order interactions** where multiple individuals interact simultaneously.

This tutorial introduces **Random Walk Hypergraph Segregation (RWHS)**, a measure that uses random walks on hypergraphs to quantify segregation patterns. RWHS captures how likely it is for nodes with similar attributes to be encountered during random explorations of the hypergraph structure.

**Key Reference:**
> Failla, A., Rossetti, G., & Cauteruccio, F. (2024). *Beyond Boundaries: Capturing Social Segregation on Hypernetworks*. International Conference on Advances in Social Networks Analysis and Mining, 40-55. Springer.

[🔝 To top](#table-of-contents)

<a id="what-is-segregation-in-hypergraphs"></a>
## What is Segregation in Hypergraphs?

In a hypergraph, segregation captures whether nodes with similar attribute values (e.g., same gender, ethnicity, affiliation) tend to participate in the same hyperedges and their local neighborhoods.

**Examples of Segregation in Hypergraphs:**

- **Email groups**: Do people from the same department tend to be in the same group emails?
- **Scientific collaborations**: Do authors from the same institution collaborate more frequently?
- **Social gatherings**: Do people with similar interests attend the same events?
- **Online communities**: Do users with similar demographics participate in the same discussions?

**High Segregation** means that nodes with similar attributes cluster together in the hypergraph structure. **Low Segregation** (high mixing) means that nodes with different attributes frequently interact.

[🔝 To top](#table-of-contents)

<a id="random-walk-hypergraph-segregation-rwhs"></a>
## Random Walk Hypergraph Segregation (RWHS)

RWHS measures segregation by performing random walks on the hypergraph and computing how often nodes with the same attribute values are encountered. The intuition is:

- If a hypergraph is **highly segregated**, random walks starting from a node will frequently encounter nodes with the same attributes.
- If a hypergraph is **well-mixed**, random walks will encounter nodes with diverse attributes.

RWHS returns a score for each node and each attribute, ranging from 0 to 1:
- **RWHS ≈ 1**: High segregation (the node mostly encounters similar nodes)
- **RWHS ≈ 0**: Low segregation (the node encounters diverse nodes)

**Two Variants:**

1. **Meet-wise RWHS**: Considers all nodes encountered in the walk
2. **Jump-wise RWHS**: Considers only consecutive pairs of nodes in the walk

[🔝 To top](#table-of-contents)

<a id="meet-wise-rwhs"></a>
### Meet-wise RWHS

**Meet-wise RWHS** measures the proportion of nodes encountered in a random walk that share the same attribute value as the starting node.

**Formula (intuitive):**
```
RWHS_meet(v, attr) = (# of visited nodes with same attr value as v) / (total # of visited nodes)
```

**When to use:**
- You want to capture the **overall composition** of the neighborhood
- You care about **any encounter** during the walk, not just direct transitions
- Example: "What fraction of people I meet in my social circles share my affiliation?"

[🔝 To top](#table-of-contents)

<a id="jump-wise-rwhs"></a>
### Jump-wise RWHS

**Jump-wise RWHS** measures the proportion of consecutive transitions in a random walk where both nodes share the same attribute value.

**Formula (intuitive):**
```
RWHS_jump(v, attr) = (# of consecutive pairs with same attr) / (total # of transitions)
```

**When to use:**
- You want to capture **direct transitions** between similar nodes
- You care about **immediate connections**, not distant encounters
- Example: "When I move from one group to another, how often do I stay within my affiliation?"

**Key Difference:**
- **Meet-wise** is more **global** (considers all nodes in the walk)
- **Jump-wise** is more **local** (considers only adjacent transitions)

[🔝 To top](#table-of-contents)

<a id="static-rwhs"></a>
## Static RWHS

Let's start with the basic `rwhs()` function that operates on a single temporal snapshot.

<a id="basic-example"></a>
### Basic Example

We'll create a small hypergraph with node attributes and compute RWHS scores.

In [ ]:
from ash_model import ASH
from ash_model.measures.hyper_segregation import rwhs, temporal_rwhs
import random
import numpy as np

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

# Create a temporal hypergraph
h = ASH()

# Add nodes with attributes (simulating a small community)
# Attribute: 'group' (A, B, or C)
node_attributes = {
    1: {'group': 'A'},
    2: {'group': 'A'},
    3: {'group': 'A'},
    4: {'group': 'B'},
    5: {'group': 'B'},
    6: {'group': 'B'},
    7: {'group': 'C'},
    8: {'group': 'C'},
}

# Add nodes with their attributes
for node, attrs in node_attributes.items():
    h.add_node(node, start=0, end=0, attr_dict=attrs)

# Add hyperedges at time 0 with varying degrees of mixing
# High segregation: mostly within-group hyperedges
h.add_hyperedge([1, 2, 3], start=0, end=0)      # All group A
h.add_hyperedge([4, 5, 6], start=0, end=0)      # All group B
h.add_hyperedge([7, 8], start=0, end=0)         # All group C
h.add_hyperedge([1, 2, 4], start=0, end=0)      # Mixed A-B
h.add_hyperedge([5, 7], start=0, end=0)         # Mixed B-C
h.add_hyperedge([3, 6, 8], start=0, end=0)      # Mixed A-B-C

print(f"Created hypergraph with {h.number_of_nodes()} nodes and {h.number_of_hyperedges()} hyperedges")
print(f"\nNode groups:")
for node, attrs in node_attributes.items():
    print(f"  Node {node}: group {attrs['group']}")

Created hypergraph with 8 nodes and 6 hyperedges

Node groups:
  Node 1: group A
  Node 2: group A
  Node 3: group A
  Node 4: group B
  Node 5: group B
  Node 6: group B
  Node 7: group C
  Node 8: group C


In [5]:
# Compute RWHS with meet-wise method
scores_meet = rwhs(h, s=1, tid=0, num_walks=100, walk_length=5, method="meet")

print("Meet-wise RWHS scores:")
print("-" * 40)
for node in sorted(scores_meet.keys()):
    group = node_attributes[node]['group']
    score = scores_meet[node]['group']
    print(f"Node {node} (group {group}): RWHS = {score:.3f}")

print("\n📊 Interpretation:")
print("  Higher scores → Node encounters more nodes from its own group")
print("  Lower scores → Node encounters diverse groups")

Meet-wise RWHS scores:
----------------------------------------
Node 1 (group A): RWHS = 0.427
Node 2 (group A): RWHS = 0.453
Node 3 (group A): RWHS = 0.385
Node 4 (group B): RWHS = 0.268
Node 5 (group B): RWHS = 0.367
Node 6 (group B): RWHS = 0.320
Node 7 (group C): RWHS = 0.207
Node 8 (group C): RWHS = 0.130

📊 Interpretation:
  Higher scores → Node encounters more nodes from its own group
  Lower scores → Node encounters diverse groups


<a id="comparing-methods"></a>
### Comparing Methods

Let's compare meet-wise and jump-wise RWHS to understand the difference.

In [6]:
# Compute both methods
scores_meet = rwhs(h, s=1, tid=0, num_walks=100, walk_length=5, method="meet")
scores_jump = rwhs(h, s=1, tid=0, num_walks=100, walk_length=5, method="jump")

print("Comparison: Meet-wise vs Jump-wise RWHS")
print("=" * 60)
print(f"{'Node':<6} {'Group':<7} {'Meet-wise':<12} {'Jump-wise':<12} {'Difference':<12}")
print("-" * 60)

for node in sorted(scores_meet.keys()):
    group = node_attributes[node]['group']
    meet_score = scores_meet[node]['group']
    jump_score = scores_jump[node]['group']
    diff = meet_score - jump_score
    
    print(f"{node:<6} {group:<7} {meet_score:<12.3f} {jump_score:<12.3f} {diff:+.3f}")

print("\n📊 Observations:")
print("  • Meet-wise considers all encounters → captures global mixing")
print("  • Jump-wise considers only transitions → captures local mixing")
print("  • Differences reveal whether segregation is global vs local")

Comparison: Meet-wise vs Jump-wise RWHS
Node   Group   Meet-wise    Jump-wise    Difference  
------------------------------------------------------------
1      A       0.435        0.625        -0.190
2      A       0.403        0.625        -0.222
3      A       0.362        0.568        -0.205
4      B       0.285        0.552        -0.268
5      B       0.372        0.585        -0.212
6      B       0.352        0.522        -0.170
7      C       0.230        0.492        -0.262
8      C       0.145        0.415        -0.270

📊 Observations:
  • Meet-wise considers all encounters → captures global mixing
  • Jump-wise considers only transitions → captures local mixing
  • Differences reveal whether segregation is global vs local


<a id="effect-of-random-walk-parameters"></a>
### Effect of Random Walk Parameters

The RWHS measure depends on random walk parameters `p` (return parameter) and `q` (in-out parameter). Let's explore their effect.

**Parameters:**
- **p**: Controls the likelihood of returning to the previous node
  - High p → less likely to return (more exploration)
  - Low p → more likely to return (more local)
- **q**: Controls the likelihood of exploring vs. staying local
  - High q → prefer nodes close to previous position
  - Low q → prefer exploring distant nodes

These parameters come from Node2Vec random walks.

In [7]:
# Test different parameter combinations
param_configs = [
    (1.0, 1.0, "Balanced (BFS-DFS)"),
    (0.5, 2.0, "More local (high q)"),
    (2.0, 0.5, "More exploratory (low q)"),
]

print("Effect of Random Walk Parameters on RWHS")
print("=" * 70)

for p, q, description in param_configs:
    scores = rwhs(h, s=1, tid=0, num_walks=100, walk_length=5, p=p, q=q, method="meet")
    
    avg_score = np.mean([scores[node]['group'] for node in scores])
    
    print(f"\n{description} (p={p}, q={q}):")
    print(f"  Average RWHS: {avg_score:.3f}")
    
    # Show a few node scores
    sample_nodes = [1, 4, 7]  # One from each group
    for node in sample_nodes:
        group = node_attributes[node]['group']
        score = scores[node]['group']
        print(f"  Node {node} (group {group}): {score:.3f}")

print("\n📊 Interpretation:")
print("  • Different parameters capture different notions of neighborhood")
print("  • Choose parameters based on the mixing scale you want to measure")

Effect of Random Walk Parameters on RWHS

Balanced (BFS-DFS) (p=1.0, q=1.0):
  Average RWHS: 0.308
  Node 1 (group A): 0.412
  Node 4 (group B): 0.300
  Node 7 (group C): 0.220

More local (high q) (p=0.5, q=2.0):
  Average RWHS: 0.308
  Node 1 (group A): 0.405
  Node 4 (group B): 0.320
  Node 7 (group C): 0.212

More exploratory (low q) (p=2.0, q=0.5):
  Average RWHS: 0.333
  Node 1 (group A): 0.448
  Node 4 (group B): 0.320
  Node 7 (group C): 0.210

📊 Interpretation:
  • Different parameters capture different notions of neighborhood
  • Choose parameters based on the mixing scale you want to measure


<a id="temporal-rwhs"></a>
## Temporal RWHS

For temporal hypergraphs, we can use **time-respecting random walks** that respect the temporal ordering of hyperedges. This is useful for analyzing segregation in dynamic networks where interactions have a temporal structure.

**Temporal RWHS** uses `temporal_rwhs()` which:
- Performs random walks on hyperedges (not nodes)
- Respects temporal constraints (can only move forward in time)
- Captures segregation in the temporal evolution

In [15]:
# Create a temporal hypergraph with evolution
h_temp = ASH()

# Add nodes with attributes across both time periods
for node, attrs in node_attributes.items():
    h_temp.add_node(node, start=0, end=1, attr_dict=attrs)

# Time 0: Initial configuration (mixed)
h_temp.add_hyperedge([1, 2, 4], eid="e1", start=0, end=0)
h_temp.add_hyperedge([3, 5], eid="e2", start=0, end=0)
h_temp.add_hyperedge([6, 7, 8], eid="e3", start=0, end=0)

# Time 1: More segregated
h_temp.add_hyperedge([1, 2, 3], eid="e4", start=1, end=1)  # Group A
h_temp.add_hyperedge([4, 5, 6], eid="e5", start=1, end=1)  # Group B
h_temp.add_hyperedge([7, 8], eid="e6", start=1, end=1)     # Group C

print("Temporal hypergraph created")
print(f"  Snapshots: {h_temp.temporal_snapshots_ids()}")
print(f"  Nodes: {h_temp.number_of_nodes()}")
print(f"  Hyperedges: {h_temp.number_of_hyperedges()}")

Temporal hypergraph created
  Snapshots: [0, 1]
  Nodes: 8
  Hyperedges: 6


In [9]:
# Compute temporal RWHS
# Note: temporal_rwhs works on hyperedge walks
print("Computing Temporal RWHS (this uses time-respecting random walks)...")

scores_temporal = temporal_rwhs(
    h_temp, 
    s=1, 
    tid=0,  # Starting from time 0
    num_walks=50, 
    walk_length=3,
    method="meet"
)

print("\nTemporal RWHS scores:")
print("-" * 40)
for node in sorted(scores_temporal.keys()):
    if node in node_attributes:
        group = node_attributes[node]['group']
        score = scores_temporal[node]['group']
        print(f"Node {node} (group {group}): RWHS = {score:.3f}")

print("\n📊 Interpretation:")
print("  Temporal RWHS captures segregation along temporal paths")
print("  Useful for understanding how groups interact over time")

Computing Temporal RWHS (this uses time-respecting random walks)...

Temporal RWHS scores:
----------------------------------------

📊 Interpretation:
  Temporal RWHS captures segregation along temporal paths
  Useful for understanding how groups interact over time


<a id="practical-examples"></a>
## Practical Examples

Let's explore some practical scenarios where RWHS is useful.

<a id="measuring-attribute-mixing"></a>
### Example 1: Measuring Attribute Mixing

Create a hypergraph and measure mixing for multiple attributes.

In [10]:
# Create a larger hypergraph with multiple attributes
h_multi = ASH()

# 20 nodes with two attributes: department and seniority
for i in range(1, 21):
    dept = 'Engineering' if i <= 10 else 'Marketing'
    seniority = 'Junior' if i % 3 == 0 else 'Senior'
    h_multi.add_node(i, start=0, end=0, attr_dict={'department': dept, 'seniority': seniority})

# Add hyperedges with some department mixing
np.random.seed(42)
for j in range(30):
    # Mix of within-department and cross-department hyperedges
    if np.random.random() < 0.7:  # 70% within department
        dept_idx = np.random.choice([0, 1])
        start_node = 1 + dept_idx * 10
        size = np.random.randint(2, 5)
        nodes = np.random.choice(range(start_node, start_node + 10), size=size, replace=False)
    else:  # 30% mixed
        size = np.random.randint(2, 5)
        nodes = np.random.choice(range(1, 21), size=size, replace=False)
    
    h_multi.add_hyperedge(list(nodes), eid=f"e{j+1}", start=0, end=0)

print(f"Created hypergraph with {h_multi.number_of_nodes()} nodes and {h_multi.number_of_hyperedges()} hyperedges")
print(f"\nNode attributes:")
print(f"  Departments: Engineering (1-10), Marketing (11-20)")
print(f"  Seniority: Junior (every 3rd node), Senior (others)")

Created hypergraph with 20 nodes and 28 hyperedges

Node attributes:
  Departments: Engineering (1-10), Marketing (11-20)
  Seniority: Junior (every 3rd node), Senior (others)


In [11]:
# Compute RWHS for both attributes
scores_multi = rwhs(h_multi, s=1, tid=0, num_walks=200, walk_length=8, method="meet")

# Analyze segregation by attribute
print("Segregation Analysis by Attribute")
print("=" * 60)

for attr in ['department', 'seniority']:
    scores_for_attr = [scores_multi[node][attr] for node in scores_multi if attr in scores_multi[node]]
    
    if scores_for_attr:
        avg_score = np.mean(scores_for_attr)
        std_score = np.std(scores_for_attr)
        
        print(f"\n{attr.upper()}:")
        print(f"  Average RWHS: {avg_score:.3f} ± {std_score:.3f}")
        print(f"  Min: {min(scores_for_attr):.3f}")
        print(f"  Max: {max(scores_for_attr):.3f}")
        
        # Classify segregation level
        if avg_score > 0.7:
            level = "HIGH segregation"
        elif avg_score > 0.5:
            level = "MODERATE segregation"
        else:
            level = "LOW segregation (well-mixed)"
        print(f"  → {level}")

print("\n📊 Interpretation:")
print("  Compare RWHS across attributes to identify which dimensions")
print("  show stronger segregation patterns")

Segregation Analysis by Attribute

DEPARTMENT:
  Average RWHS: 0.781 ± 0.069
  Min: 0.577
  Max: 0.869
  → HIGH segregation

SENIORITY:
  Average RWHS: 0.483 ± 0.172
  Min: 0.176
  Max: 0.666
  → LOW segregation (well-mixed)

📊 Interpretation:
  Compare RWHS across attributes to identify which dimensions
  show stronger segregation patterns


<a id="comparing-segregation-across-time"></a>
### Example 2: Comparing Segregation Across Time

Track how segregation evolves over time.

In [12]:
# Create a hypergraph that evolves from mixed to segregated
h_evolution = ASH()

# Add nodes with attributes across all time periods
for i in range(1, 13):
    group = 'A' if i <= 4 else ('B' if i <= 8 else 'C')
    h_evolution.add_node(i, start=0, end=2, attr_dict={'group': group})

# Time 0: Well-mixed (low segregation)
mixed_edges = [
    [1, 5, 9], [2, 6, 10], [3, 7, 11], [4, 8, 12],  # Mixed groups
    [1, 2, 5], [3, 6, 9], [4, 7, 10]                # More mixing
]
for idx, nodes in enumerate(mixed_edges):
    h_evolution.add_hyperedge(nodes, eid=f"t0_e{idx+1}", start=0, end=0)

# Time 1: Moderate segregation
moderate_edges = [
    [1, 2, 3], [5, 6, 7], [9, 10, 11],  # Within-group
    [1, 5], [4, 8], [7, 11]              # Some mixing
]
for idx, nodes in enumerate(moderate_edges):
    h_evolution.add_hyperedge(nodes, eid=f"t1_e{idx+1}", start=1, end=1)

# Time 2: High segregation
segregated_edges = [
    [1, 2, 3, 4],        # All A
    [5, 6, 7, 8],        # All B
    [9, 10, 11, 12],     # All C
]
for idx, nodes in enumerate(segregated_edges):
    h_evolution.add_hyperedge(nodes, eid=f"t2_e{idx+1}", start=2, end=2)

print("Created temporal hypergraph with evolving segregation")
print(f"  Snapshots: {h_evolution.temporal_snapshots_ids()}")

Created temporal hypergraph with evolving segregation
  Snapshots: [0, 1, 2]


In [13]:
# Compute RWHS at each time point
print("Segregation Evolution Over Time")
print("=" * 60)

time_scores = []
for t in range(3):
    scores_t = rwhs(h_evolution, s=1, tid=t, num_walks=100, walk_length=6, method="meet")
    
    # Compute average RWHS
    group_scores = [scores_t[node]['group'] for node in scores_t if 'group' in scores_t[node]]
    avg_score = np.mean(group_scores) if group_scores else 0
    time_scores.append(avg_score)
    
    print(f"\nTime {t}:")
    print(f"  Average RWHS: {avg_score:.3f}")
    print(f"  Nodes with RWHS > 0.5: {sum(1 for s in group_scores if s > 0.5)}/{len(group_scores)}")
    
    # Sample nodes
    sample_nodes = [1, 5, 9]  # One from each group
    for node in sample_nodes:
        if node in scores_t:
            score = scores_t[node]['group']
            print(f"    Node {node}: {score:.3f}")

print("\n📊 Trend Analysis:")
for i in range(1, len(time_scores)):
    change = time_scores[i] - time_scores[i-1]
    direction = "↑ increased" if change > 0 else "↓ decreased"
    print(f"  t{i-1} → t{i}: RWHS {direction} by {abs(change):.3f}")

print("\n  → The hypergraph became more segregated over time")

Segregation Evolution Over Time

Time 0:
  Average RWHS: 0.162
  Nodes with RWHS > 0.5: 0/12
    Node 1: 0.268
    Node 5: 0.116
    Node 9: 0.096

Time 1:
  Average RWHS: 0.508
  Nodes with RWHS > 0.5: 5/9
    Node 1: 0.408
    Node 5: 0.408
    Node 9: 0.596

Time 2:
  Average RWHS: 0.789
  Nodes with RWHS > 0.5: 12/12
    Node 1: 0.768
    Node 5: 0.794
    Node 9: 0.798

📊 Trend Analysis:
  t0 → t1: RWHS ↑ increased by 0.346
  t1 → t2: RWHS ↑ increased by 0.281

  → The hypergraph became more segregated over time


<a id="parameter-tuning"></a>
### Example 3: Parameter Tuning

Understanding how walk length and number of walks affect RWHS.

In [8]:
# Create a temporal hypergraph with evolution
h_temp = ASH()

# Add nodes with attributes across both time periods
for node, attrs in node_attributes.items():
    h_temp.add_node(node, start=0, end=1, attr_dict=attrs)

# Time 0: Initial configuration (mixed)
h_temp.add_hyperedge([1, 2, 4], eid="e1", start=0, end=0)
h_temp.add_hyperedge([3, 5], eid="e2", start=0, end=0)
h_temp.add_hyperedge([6, 7, 8], eid="e3", start=0, end=0)

# Time 1: More segregated
h_temp.add_hyperedge([1, 2, 3], eid="e4", start=1, end=1)  # Group A
h_temp.add_hyperedge([4, 5, 6], eid="e5", start=1, end=1)  # Group B
h_temp.add_hyperedge([7, 8], eid="e6", start=1, end=1)     # Group C

print("Temporal hypergraph created")
print(f"  Snapshots: {h_temp.temporal_snapshots_ids()}")
print(f"  Nodes: {h_temp.number_of_nodes()}")
print(f"  Hyperedges: {h_temp.number_of_hyperedges()}")

Temporal hypergraph created
  Snapshots: [0, 1]
  Nodes: 8
  Hyperedges: 6


## Summary

Random Walk Hypergraph Segregation (RWHS) provides a powerful framework for measuring segregation in higher-order interactions.

**Key Takeaways:**

1. **Two Methods:**
   - **Meet-wise**: Global mixing (considers all encounters)
   - **Jump-wise**: Local mixing (considers consecutive transitions)

2. **Parameters:**
   - **num_walks**: More walks → more stable estimates
   - **walk_length**: Longer walks → broader neighborhoods
   - **p, q**: Control exploration vs. exploitation (from Node2Vec)
   - **s**: Minimum overlap for hyperedge connections

3. **Temporal Analysis:**
   - Use `temporal_rwhs()` for time-respecting random walks
   - Track segregation evolution over time
   - Compare segregation across different temporal windows

4. **Applications:**
   - Measure mixing across multiple attributes
   - Compare segregation levels between groups
   - Identify temporal trends in segregation
   - Evaluate interventions designed to promote diversity

**When to Use RWHS:**
- ✅ Analyzing group interactions in collaborative networks
- ✅ Measuring diversity in team formations
- ✅ Studying temporal evolution of mixing patterns
- ✅ Comparing segregation across different attributes

**Next Steps:**
- Explore the [walks tutorial](02-walks.ipynb) for understanding random walks
- Check the [attribute analysis tutorial](01-attribute_analysis.ipynb) for other attribute-based measures
- See the [multiego tutorial](05-multiego.ipynb) for group-centric analysis

[🔝 To top](#table-of-contents)

<a id="references"></a>
## References

**Main Reference:**
- Failla, A., Rossetti, G., & Cauteruccio, F. (2024). *Beyond Boundaries: Capturing Social Segregation on Hypernetworks*. In International Conference on Advances in Social Networks Analysis and Mining (pp. 40-55). Springer.

**Related Work:**
- Node2Vec random walks: Grover, A., & Leskovec, J. (2016). *node2vec: Scalable feature learning for networks*. KDD.
- Hypergraph theory: Berge, C. (1984). *Hypergraphs: combinatorics of finite sets*. North-Holland.

[🔝 To top](#table-of-contents)